<img src="images/Picture0.png" width=200x />

# Notebook 07 - Logical Design

### Covered in this notebook:
* Modeling and algorithm

### Credits:
* [Gurobi modeling examples](https://gurobi.github.io/modeling-examples/): [Logical design](https://gurobi.github.io/modeling-examples/logical_design/)
* The example below is taken from the Gurobi modeling examples Github, which took the example in turn from Model Building in Mathematical Programming Fifth Edition by Paul H. Williams.  No credit is claimed for text, code, or algorithm.  Alterations occur in phrasing, order of information, and slight tweaks to written code.

## Introduction

The following interesting example from the [Gurobi model library](https://gurobi.github.io/modeling-examples/) applies our acquired skills to a new context.  A <i>logical circuit</i> is a system with a given number of inputs and one output.  When impulses are applied to the inputs of the circuit, it responds by giving either an output (signal 1) or no output (signal 0).  The input impulses are characterized in the same way as the inputs: positive input (signal 1) or no input (signal 0).

In this example, a logical circuit is to be built using only <i>NOR gates</i>.  A NOR gate is a device with two inputs and one output, and which has the property that there is a positive output (signal 1) if and only if neither input is positive.  By connecting such gates in combination, it is possible to construct a circuit to perform any desired logical function.

For example, the circuit illustrated in the following figure will respond to the inputs A and B in the way indicated by the truth table.  (Unlabeled inputs are 0.)

<img src="images/N07_images/circuit.png"/>

<strong>The objective</strong> is to construct a circuit using the minimum number of NOR gates that will perform the logical function specified by the following truth table:

| A | B | Output |
| --- | --- | --- | 
| 0 | 0 | 0 |
| 0 | 1 | 1 |
| 1 | 0 | 1 |
| 1 | 1 | 0 |

Each output from a NOR gate must lead into exactly one input into another NOR gate.  It may be assumed that the optimal design is a "subnet" of the "maximal" net shown in the following figure.

<img src="images/N07_images/subnet.png"/>

## Model formulation

Begin brainstorming your model formulation.  Hints to one solution are included below, suggesting:

* The input data required by your program
* The decision variables of the problem
* Supplementary variables to parametrize the problem* constraints
* The objective function of the problem

Feel free to check your model against this framework before coding.

### Input data:

We will need to define some input data for the model, including:

* The values taken by A and B in each row of the truth table
* Which gates are present
* Which gates input into which other gates

### Decision variables:

The model depends on the following variables:

* Is a given gate selected as part of the system?
* Is input A an input into a given gate?  Is input B?

### Supplementary variables:

The following variable will also be useful to define, to create our framework:

* The output of a given gate

### Constraints:

We are subject to the following constraints:

* Mechanical constraints (defining the system's functionality):
    * Each gate takes at most two inputs between A, B, and output from other gates.  (If only one input is given to a gate, the other input is 0.)
    * Each gate gives exactly one output.
    * A gate's output is 1 if and only if both inputs are 0.
* Programmatic constraints:
    * The output of gate 1 must be according to the truth table.
    * The solution must be nontrivial (at least one gate must be active).

## Solution:

### Input data:

$i \in \text{Gates}=\{1,...,7\}$

$i \in \text{Gates47}=\{4,...,7\}$

$r \in \text{Rows}=\{1,...,4\}$

$\text{valueA}_r \in \{0,1 \}$: Value of the external input A in row  $r$ of the truth table.

$\text{valueB}_r \in \{0,1 \}$: Value of the external input B in row  $r$ of the truth table.


<i>Note: The Gurobipy [multidict function](https://support.gurobi.com/hc/en-us/articles/17307437899025-Tutorial-Getting-Started-with-the-Gurobi-Python-API-using-dictionaries) allows users to create multiple dictionaries simultaneously:</i>

```
keys, dictA, dictB, dictC = gp.multidict({
    key1: [valA1, valB1, valC1],
    ...
    keyn: [valAn, valBn, valCn]
})
```

<i>The `multidict` function splits a given dictionary whose values are lists of equal length into multiple dictionaries, along with a list of keys.  Our example above creates three dictionaries, namely `dictA`, `dictB`, and `dictC`, and creates a list of keys assigned to `keys`.</i>

In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
gates = ['1','2','3','4','5','6','7']

gates47 = ['4','5','6','7']

rows = ['1','2','3','4']

rows, valueA, valueB = gp.multidict({
    1: [0, 0],
    2: [0, 1],
    3: [1, 0],
    4: [1, 1]
})

### Model deployment

In [3]:
model = gp.Model('logicalDesign')

Set parameter Username


### Variables

#### Decision variables:

$\text{NOR}_{i} \in \{0,1 \}$: This binary variable is equal to 1 if NOR gate $i$  is selected, and 0 otherwise.

$\text{inputA}_{i} \in \{0,1 \}$: This binary variable is equal to 1 if external input A is an input to NOR gate $i$ , and 0 otherwise.

$\text{inputB}_{i} \in \{0,1 \}$: This binary variable is equal to 1 if external input B is an input to NOR gate $i$ , and 0 otherwise.

#### Supplementary variables:

$\text{output}_{i,r} \in \{0,1 \}$: This binary variable is the output from gate $i$ for the combination of external input signals specified  in row $r$  of the truth table.

In [4]:
NOR = model.addVars(gates, vtype=GRB.BINARY, name="NORgate" )

inputA = model.addVars(gates, vtype=GRB.BINARY, name="inputA")
inputB = model.addVars(gates, vtype=GRB.BINARY, name="inputB")

output = model.addVars(gates, rows, vtype=GRB.BINARY, name="output")

### Constraints

#### Mechanical constraints:

**External input**: A NOR gate can only have an external input if it exists.

\begin{equation}
\text{NOR}_{i} \geq \text{inputA}_{i}  \quad \forall i \in \text{Gates}
\end{equation}

\begin{equation}
\text{NOR}_{i} \geq \text{inputB}_{i}  \quad \forall i \in \text{Gates}
\end{equation}

**NOR gates**:If a NOR gate has one (or two) external inputs leading into it, only one (or no) NOR gates can feed into it. 

\begin{equation}
\text{NOR}_{2} + \text{NOR}_{3} + \text{inputA}_{1} + \text{inputB}_{2} \leq 2
\end{equation}

\begin{equation}
\text{NOR}_{4} + \text{NOR}_{5} + \text{inputA}_{2} + \text{inputB}_{2} \leq 2
\end{equation}

\begin{equation}
\text{NOR}_{6} + \text{NOR}_{7} + \text{inputA}_{3} + \text{inputB}_{3} \leq 2
\end{equation}

**Output signals**: The output signal from NOR gate $i$ must be the correct logical function (NOR) of the input signals into gate $i$, if gate $i$ exists. 

\begin{equation}
\text{output}_{2,r} + \text{output}_{1,r} \leq 1 \quad \forall r \in \text{Rows}
\end{equation}

\begin{equation}
\text{output}_{3,r} + \text{output}_{1,r} \leq 1 \quad \forall r \in \text{Rows}
\end{equation}

\begin{equation}
\text{valueA}_{i,r}*\text{inputA}_{i} + \text{output}_{i,r} \leq 1 \quad \forall i \in \text{Gates}, r \in \text{Rows}
\end{equation}

\begin{equation}
\text{valueB}_{i,r}*\text{inputB}_{i} + \text{output}_{i,r} \leq 1 \quad \forall i \in \text{Gates}, r \in \text{Rows}
\end{equation}

\begin{equation}
\text{valueA}_{i,r}*\text{inputA}_{i} + \text{valueB}_{i,r}*\text{inputB}_{i} + 
\text{output}_{i,r} - \text{NOR}_{i}  \geq 0 \quad \forall i \in \text{G47}, r \in \text{Rows}
\end{equation}

\begin{equation}
\text{valueA}_{1,r}*\text{inputA}_{1} + \text{valueB}_{1,r}*\text{inputB}_{1} + 
\text{output}_{2,r} + \text{output}_{3,r} + \text{output}_{1,r} - \text{NOR}_{1}  \geq 0 
\quad \forall r \in \text{Rows}
\end{equation}

\begin{equation}
\text{valueA}_{2,r}*\text{inputA}_{2} + \text{valueB}_{2,r}*\text{inputB}_{2} + 
\text{output}_{4,r} + \text{output}_{5,r} + \text{output}_{2,r} - \text{NOR}_{2}  \geq 0 
\quad \forall r \in \text{Rows}
\end{equation}

\begin{equation}
\text{valueA}_{3,r}*\text{inputA}_{3} + \text{valueB}_{3,r}*\text{inputB}_{3} + 
\text{output}_{6,r} + \text{output}_{7,r} + \text{output}_{3,r} - \text{NOR}_{3}  \geq 0 
\quad \forall r \in \text{Rows}
\end{equation}

Note that the two constraints above define several pieces of structural information:
* Which gates input into which other gates
* Each gate takes at most two inputs between A, B, and output from other gates.  (If only one input is given to a gate, the other input is 0.)

In [5]:
# External inputs constraints
externalInputsA = model.addConstrs( ( NOR[i] >= inputA[i]  for i in gates), name='externalInputsA')
externalInputsB = model.addConstrs( ( NOR[i] >= inputB[i]  for i in gates), name='externalInputsB')

# Gate and output signals constraints
gateOutput = model.addConstrs( (NOR[i] - output[i, r] >= 0 for i in gates for r in rows) , name='gateOutput')

# NOR gates constraints
NORgate1 = model.addConstr(NOR['2'] + NOR['3'] + inputA['1'] + inputB['1'] <= 2, name='NORgate1')
NORgate2 = model.addConstr(NOR['4'] + NOR['5'] + inputA['2'] + inputB['2'] <= 2, name='NORgate2')
NORgate3 = model.addConstr(NOR['6'] + NOR['7'] + inputA['3'] + inputB['3'] <= 2, name='NORgate3')

In [6]:
# Output signal constraint
outputSignals1_1 = model.addConstrs( (output['2',r] + output['1',r] <= 1 for r in rows), name='outputSignals1_1' )
outputSignals1_2 = model.addConstrs( (output['3',r] + output['1',r] <= 1 for r in rows), name='outputSignals1_2' )
outputSignals2_1 = model.addConstrs( (output['4',r] + output['2',r] <= 1 for r in rows), name='outputSignals2_1' )
outputSignals2_2 = model.addConstrs( (output['5',r] + output['2',r] <= 1 for r in rows), name='outputSignals2_2' )
outputSignals3_1 = model.addConstrs( (output['6',r] + output['3',r] <= 1 for r in rows), name='outputSignals3_1' )
outputSignals3_2 = model.addConstrs( (output['7',r] + output['3',r] <= 1 for r in rows), name='outputSignals3_2' )


outputSignals4 = model.addConstrs( (valueA[r]*inputA[i] + output[i,r] <= 1 for i in gates for r in rows), name='outputSignals4')
outputSignals5 = model.addConstrs( (valueB[r]*inputB[i] + output[i,r] <= 1 for i in gates for r in rows), name='outputSignals5')
outputSignals6 = model.addConstrs( (valueA[r]*inputA[i] + valueB[r]*inputB[i] + output[i,r] - NOR[i] >= 0 
                                    for i in gates for r in rows if i in gates47), name='outputSignals6')
outputSignals7 = model.addConstrs( (valueA[r]*inputA['1'] + valueB[r]*inputB['1'] 
                                    + output['2',r] + output['3',r] + output['1',r] - NOR['1'] >= 0
                                    for i in gates for r in rows), name='outputSignals7')
outputSignals8 = model.addConstrs( (valueA[r]*inputA['2'] + valueB[r]*inputB['2'] 
                                    + output['4',r] + output['5',r] + output['2',r] - NOR['2'] >= 0
                                    for i in gates for r in rows), name='outputSignals8')
outputSignals9 = model.addConstrs( (valueA[r]*inputA['3'] + valueB[r]*inputB['3'] 
                                    + output['6',r] + output['7',r] + output['3',r] - NOR['3'] >= 0
                                    for i in gates for r in rows), name='outputSignals9')

#### Programmatic constraints:

**Gate 1**: For NOR gate 1, the output variables are fixed at the values specified in the truth table. 

\begin{equation}
\text{output}_{1,1} = 0, \text{output}_{1,2} = 1, \text{output}_{1,3} = 1,  \text{output}_{1,4} = 0   
\end{equation}

**Trivial solution**: To avoid a trivial solution containing no NOR gates, it is necessary to impose a constraint that selects NOR gate 1.

\begin{equation}
\text{NOR}_{1} \geq 1  
\end{equation}

In [7]:
# For NOR gate 1, the output variables are fixed at the values specified in the truth table.
output['1',1].ub = 0
output['1',2].lb = 1
output['1',3].lb = 1
output['1',4].ub = 0

# In order to avoid a trivial solution containing no NOR gates, it is necessary to impose a constraint 
# that selects NOR gate 1.
NOR['1'].lb = 1

### Objective function

**Number of gates**: The objective is to minimize the number of NOR gates selected.

\begin{equation}
\text{Minimize} \quad  \sum_{i \in \text{Gates}} \text{NOR}_{i}
\end{equation}

In [8]:
# Objective function.
model.setObjective(NOR.sum(), GRB.MINIMIZE)

In [9]:
# Verify model formulation
model.write('logicalDesign.lp')

# Run optimization engine
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 225 rows, 49 columns and 696 nonzeros
Model fingerprint: 0xfb7e2b7f
Variable types: 0 continuous, 49 integer (49 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 225 rows and 49 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 5 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+00, best bound 5.000000000000e+00, gap 0.0000%


In [10]:
model.write('logicalDesign.sol')

In [17]:
# Output reports
print("\n\n_________________________________________________________________________________")
print(f"The optimal circuit design:")
print("_________________________________________________________________________________")
for i in gates:
    if (NOR[i].x > 0.5):
        if (inputA[i].x + inputB[i].x > 0.5):
            print("NOR gate", i, "is active.  Input A {incA}, and Input B {incB}, an input to the gate."
                  .format(incA = 'is' if inputA[i].x else 'is not', incB = 'is' if inputB[i].x else 'is not'))
        else:
            print(f"NOR gate {i} is active.")





_________________________________________________________________________________
The optimal circuit design:
_________________________________________________________________________________
NOR gate 1 is active.
NOR gate 2 is active.  Input A is, and Input B is, an input to the gate.
NOR gate 3 is active.
NOR gate 6 is active.  Input A is not, and Input B is, an input to the gate.
NOR gate 7 is active.  Input A is, and Input B is not, an input to the gate.
